## Introduction

In this assignment, you are going to build a classifier for named entities from the Groningen Meaning Bank corpus.  Named entity recognition (NER) takes noun phrases from a text and identifies whether they are persons, organizations, and so on.  You will be using the Groningen Meaning Bank named entity corpus available on mltgpu at `/scratch/lt2222-v21-resources/GMB_dataset.txt`.  In this version of the task, you will assume we know *that* something is a named entity, and instead use multi-class classification to identify its type.  So you will be doing named entity classification but *not* recognition.

The data looks like this: 

```
3996    182.0   Nicole  NNP     B-per
3997    182.0   Ritchie NNP     I-per
3998    182.0   is      VBZ     O
3999    182.0   pregnant        JJ      O
4000    182.0   .       .       O
4001    183.0   Speaking        VBG     O
4002    183.0   to      TO      O
4003    183.0   ABC     NNP     B-org
4004    183.0   News    NNP     I-org
4005    183.0   interviewer     NN      O
4006    183.0   Dianne  NNP     B-per
4007    183.0   Sawyer  NNP     I-per
4008    183.0   ,       ,       O
4009    183.0   the     DT      O
4010    183.0   25-year-old     JJ      O
4011    183.0   co-star NN      O
4012    183.0   of      IN      O
4013    183.0   TV      NN      O
4014    183.0   's      POS     O
4015    183.0   The     DT      B-art
4016    183.0   Simple  NNP     I-art
4017    183.0   Life    NNP     I-art
4018    183.0   said    VBD     O
4019    183.0   she     PRP     O
4020    183.0   is      VBZ     O
4021    183.0   almost  RB      O
4022    183.0   four    CD      O
4023    183.0   months  NNS     O
4024    183.0   along   IN      O
4025    183.0   in      IN      O
4026    183.0   her     PRP$    O
4027    183.0   pregnancy       NN      O
4028    183.0   .       .       O
```

The first column is the line number.  The second column is a sentence number (for some reason given as a float; ignore it).  The third column is the word.  The fourth column is a part of speech (POS) tag in Penn Treebank format.  The last column contains the named entity annotation. 

The annotation works like this.  Every `O` just means that the row does not represent a named entity.  `B-xyx` means the first word in a named entity with type `xyx`. `I-xyz` means the second and later words of an `xyz` entity, if there are any.  That means that every time there's a `B` or an `I`, there's a named entity.  

The entity types in the corpus are `art`,
`eve`,
`geo`,
`gpe`,
`nat`,
`org`,
`per`,
and `tim`

Your task is the following.
1. To preprocess the text (lowercase and lemmatize; punctuation can be preserved as it gets its own rows).
2. To create instances from every from every identified named entity in the text with the type of the NE as the class, and a surrounding context of five words on either side as the features.  
3. To generate vectors and split the instances into training and testing datasets at random.
4. To train a support vector machine (via `sklearn.svm.LinearSVC`) for classifying the NERs.
5. To evaluate the performance of the classifier.

You will do this by modifying a separate file containing functions that will be called from this notebook as a module.  You can modify this notebook for testing purposes but please only submit the original.  You will document everything in Markdown in README.md and submit a GitHub repository URL.

This assignment is due on **Tuesday, 2021 March 9 at 23:59**.  It has **25 points** and **7 bonus points**.

In [1]:
import a2
from sklearn.svm import LinearSVC

In [2]:
gmbfile = open('/scratch/lt2222-v21-resources/GMB_dataset.txt', "r")

## Part 1 - preprocessing (3 points)

See step 1 above.  The data is coming to you as an unused file handle object.  You can return the data in any indexable form you like.  You can also choose to remove infrequent or uninformative words to reduce the size of the feature space. (Document this in README.md.)

In [3]:
inputdata = a2.preprocess(gmbfile)
gmbfile.close()
inputdata[20:40]

['19\t1.0\ttroops\tNNS\tO\n',
 '20\t1.0\tfrom\tIN\tO\n',
 '21\t1.0\tthat\tDT\tO\n',
 '22\t1.0\tcountry\tNN\tO\n',
 '23\t1.0\t.\t.\tO\n',
 '24\t2.0\tFamilies\tNNS\tO\n',
 '25\t2.0\tof\tIN\tO\n',
 '26\t2.0\tsoldiers\tNNS\tO\n',
 '27\t2.0\tkilled\tVBN\tO\n',
 '28\t2.0\tin\tIN\tO\n',
 '29\t2.0\tthe\tDT\tO\n',
 '30\t2.0\tconflict\tNN\tO\n',
 '31\t2.0\tjoined\tVBD\tO\n',
 '32\t2.0\tthe\tDT\tO\n',
 '33\t2.0\tprotesters\tNNS\tO\n',
 '34\t2.0\twho\tWP\tO\n',
 '35\t2.0\tcarried\tVBD\tO\n',
 '36\t2.0\tbanners\tNNS\tO\n',
 '37\t2.0\twith\tIN\tO\n',
 '38\t2.0\tsuch\tJJ\tO\n']

## Part 2 - Creating instances (7 points)

Do step 2 above.  You will create a collection of Instance objects.  Remember to consider the case where the NE is at the beginning of a sentence or at the end, or close to either (you can create a special start token for that).  You can also start counting from before the `B` end of the NE mention and after the last `I` of the NE mention. That means that the instances should include things before and after the named entity mention, but not the named entity text itself.

In [4]:
instances = a2.create_instances(inputdata)

In [5]:
instances[20:40]

[Class: eve Features: ['something', 'something', 'something', 'something'],
 Class: org Features: ['something', 'something'],
 Class: gpe Features: ['something', 'something', 'something', 'something', 'something', 'something', 'something'],
 Class: per Features: ['something', 'something'],
 Class: tim Features: ['something', 'something', 'something', 'something', 'something'],
 Class: gpe Features: ['something', 'something', 'something', 'something', 'something', 'something'],
 Class: per Features: ['something', 'something', 'something', 'something'],
 Class: org Features: ['something', 'something', 'something', 'something', 'something', 'something', 'something'],
 Class: per Features: ['something'],
 Class: geo Features: ['something', 'something', 'something', 'something', 'something', 'something', 'something', 'something', 'something', 'something'],
 Class: per Features: ['something', 'something', 'something', 'something', 'something', 'something', 'something'],
 Class: per Features:

## Part 3 - Creating the table and splitting (10 points)

Here you're going to write the functions that create a data table with "document" vectors representing each instance and split the table into training and testing sets and random with an 80%/20% train/test split.

In [6]:
bigdf = a2.create_table(instances)
bigdf[20:40]

,class,0,1,2,3,4,5,6,7,8,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
20,eve,0.655986,0.415185,0.059694,0.707719,0.322907,0.194757,0.277327,0.895893,0.017205,...,0.974347,0.858885,0.995533,0.372458,0.687126,0.187404,0.434756,0.832648,0.287794,0.477700
21,nat,0.303020,0.621403,0.818070,0.583170,0.481382,0.134504,0.760761,0.556372,0.559527,...,0.023771,0.696146,0.326242,0.861926,0.268099,0.309540,0.724976,0.554450,0.737834,0.213573
22,gpe,0.842124,0.947598,0.176131,0.880908,0.947319,0.989095,0.056913,0.599464,0.493132,...,0.988192,0.923232,0.395368,0.273921,0.160180,0.096286,0.807849,0.662150,0.631081,0.549078
23,geo,0.738763,0.548412,0.324128,0.481976,0.907487,0.573559,0.441798,0.568445,0.426464,...,0.474025,0.337602,0.646068,0.611336,0.139842,0.171521,0.560293,0.662646,0.948103,0.125349
24,geo,0.672174,0.462499,0.788121,0.937968,0.077253,0.912724,0.604421,0.520988,0.395650,...,0.349827,0.121361,0.579793,0.424547,0.284203,0.488576,0.021130,0.955238,0.425942,0.934088
25,org,0.190210,0.396298,0.785996,0.305488,0.869708,0.045089,0.125519,0.671604,0.902194,...,0.833199,0.630202,0.240270,0.084578,0.083816,0.211891,0.745035,0.166060,0.050051,0.350905
26,org,0.570826,0.530055,0.278830,0.731944,0.787163,0.202206,0.924931,0.592754,0.436697,...,0.848131,0.314189,0.752178,0.656124,0.576766,0.028159,0.692283,0.411886,0.787077,0.093095
27,nat,0.480336,0.382422,0.490689,0.616319,0.315667,0.476534,0.353519,0.719981,0.770045,...,0.088550,0.939790,0.662498,0.578622,0.440192,0.805044,0.918339,0.600827,0.212466,0.944630
28,org,0.176863,0.256072,0.391187,0.222540,0.085265,0.462145,0.235633,0.639741,0.742303,...,0.437307,0.301980,0.800389,0.707553,0.107649,0.983342,0.048044,0.172435,0.980198,0.489569
29,gpe,0.637363,0.771508,0.856566,0.582754,0.858660,0.619299,0.511857,0.655012,0.316246,...,0.800751,0.477166,0.684987,0.625954,0.715216,0.430617,0.683785,0.284774,0.612989,0.547428


In [7]:
train_X, train_y, test_X, test_y = a2.ttsplit(bigdf)

# X and y mean feature matrix and class respectively.
train_X, train_y, test_X, test_y

(array([[0.99636253, 0.7455104 , 0.72385553, ..., 0.03679133, 0.04820846,
         0.66081946],
        [0.39568598, 0.37229523, 0.17353789, ..., 0.00196175, 0.77350708,
         0.3204305 ],
        [0.11010834, 0.37748122, 0.18390218, ..., 0.09508813, 0.37348675,
         0.60344735],
        ...,
        [0.01200433, 0.87604778, 0.77053487, ..., 0.11399781, 0.65884565,
         0.33744048],
        [0.07136329, 0.36089497, 0.62061408, ..., 0.40636414, 0.15702171,
         0.09905803],
        [0.17420434, 0.75884942, 0.88563346, ..., 0.70899009, 0.41493292,
         0.4479619 ]]), 0     art
 1     nat
 2     org
 3     eve
 4     tim
      ... 
 75    per
 76    gpe
 77    tim
 78    gpe
 79    geo
 Name: class, Length: 80, dtype: object, array([[0.55690157, 0.58498515, 0.43289855, ..., 0.70384379, 0.00780216,
         0.26405441],
        [0.90644824, 0.84436608, 0.16041137, ..., 0.24061262, 0.23453932,
         0.19677123],
        [0.20121931, 0.52836921, 0.18535695, ..., 0.42235

In [8]:
len(test_y) / (len(test_y) + len(train_y))

0.2

In [9]:
len(test_X) / (len(test_X) + len(train_X))

0.2

In [10]:
test_y[0]

'per'

## Part 4 - Training the model (0 points)

This part you won't do yourself.

In [11]:
model = LinearSVC()
model.fit(train_X, train_y)
train_predictions = model.predict(train_X)
test_predictions = model.predict(test_X)

In [12]:
train_predictions

array(['art', 'nat', 'org', 'eve', 'tim', 'gpe', 'art', 'per', 'org',
       'eve', 'gpe', 'tim', 'geo', 'geo', 'nat', 'nat', 'gpe', 'org',
       'nat', 'nat', 'nat', 'nat', 'per', 'tim', 'nat', 'per', 'per',
       'org', 'eve', 'tim', 'org', 'geo', 'per', 'tim', 'geo', 'art',
       'art', 'tim', 'gpe', 'gpe', 'per', 'org', 'eve', 'eve', 'geo',
       'nat', 'eve', 'org', 'org', 'org', 'per', 'gpe', 'tim', 'tim',
       'geo', 'tim', 'org', 'per', 'org', 'nat', 'art', 'eve', 'org',
       'nat', 'per', 'tim', 'per', 'eve', 'per', 'art', 'geo', 'tim',
       'geo', 'geo', 'eve', 'per', 'gpe', 'tim', 'gpe', 'geo'],
      dtype=object)

In [13]:
train_y

0     art
1     nat
2     org
3     eve
4     tim
     ... 
75    per
76    gpe
77    tim
78    gpe
79    geo
Name: class, Length: 80, dtype: object

In [14]:
test_predictions

array(['org', 'geo', 'nat', 'per', 'eve', 'org', 'org', 'nat', 'nat',
       'org', 'nat', 'org', 'per', 'tim', 'nat', 'tim', 'org', 'eve',
       'nat', 'nat'], dtype=object)

In [15]:
test_y

0     per
1     art
2     art
3     eve
4     art
5     tim
6     geo
7     tim
8     org
9     art
10    per
11    art
12    org
13    nat
14    org
15    art
16    geo
17    org
18    art
19    art
Name: class, dtype: object

## Part 5 - Evaluation (5 points)

Investigate for yourself what a "confusion matrix".  Then implement a function that takes the data and produces a confusion matrix in any readable form that allows us to compare the performance of the model by class.  

In [16]:
a2.confusion_matrix(test_y, test_predictions)

I'm confusing.


"I'm confused."

In [17]:
a2.confusion_matrix(train_y, train_predictions)

I'm confusing.


"I'm confused."

Examine the matrix and describe your observations in README.md.  In particular, what do you notice about the predictions on the training data compared to those on the test data.

## Bonus Part A - Error analysis (2 points)

Look at the weakest-performing classes in the confusion matrix (or any, if they all perform poorly to the same extent).  Find some examples in the test data on which the classifier classified incorrectly for those classes.  What do you think is the reason why those are hard?  Consider linguistic factors and statistical factors, if applicable.  Write your answer in README.md.

## Bonus Part B - Expanding the feature space (7 points)

Run the entire process above, but incorporate part-of-speech tag information into the feature vectors.  It's your choice as to how to do this, but document it in README.md.  Your new process should run from the single call below:

In [18]:
a2.bonusb('/scratch/lt2222-v21-resources/GMB_dataset.txt')